In [ ]:
from ipyleaflet import Marker, basemaps
from ipyleaflet.velocity import Velocity
import geemap
import utils.wind_data as wd

import xarray as xr
import cdsapi
c = cdsapi.Client()

In [ ]:
c.retrieve(
    'satellite-land-cover',
    {
        'variable': 'all',
        'format': 'zip',
        'year': '2020',
        'version': 'v2.1.1',
    },
    'download.zip')

In [ ]:
nc_file = 'data/nc_files/C3S-LC-L4-LCCS-Map-300m-P1Y-2020-v2.1.1.nc'

In [ ]:
ds = xr.open_dataset(nc_file)
print(ds)

In [ ]:
ds.dims

In [ ]:
ds.coords

In [ ]:
ds.attrs

In [ ]:
ds['lccs_class'].shape

In [ ]:
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
        ],
        'year': '2020',
        'month': '08',
        'day': '27',
        'time': '12:00',
        'grid': "2.5/2.5",
    },
    'data/nc_files/wind_2020_08_27_12.nc',
)

In [ ]:
ds = xr.open_dataset('data/nc_files/wind_2020_08_27_12.nc')
print(ds)

In [ ]:
ds = wd.reshape_data(ds)

In [ ]:
center = (44.304655, 5.506011)
zoom = 5

my_map = geemap.Map(center=center,
    zoom=zoom,
    interpolation='nearest',
    basemap=basemaps.CartoDB.DarkMatter
)

display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data'
}

# Add winds to the map
wind = Velocity(data=ds,
                zonal_speed='u10',
                meridional_speed='v10',
                latitude_dimension='latitude',
                longitude_dimension='longitude',
                velocity_scale=0.01,
                max_velocity=20,
                display_options=display_options)
my_map.add_layer(wind)
my_map.addLayerControl()

In [ ]:
marker = Marker(location=center, draggable=False, title="Test marker")
my_map.add_layer(marker)
my_map.addLayerControl()

In [ ]:
my_map

In [ ]:
# save map to html
my_map.save('output/maps/wind_2020_08_27.html')